In [1]:
"""Build Baseline Location Classifier
Approaches:
- instance representation in form of 'bag' of words 
    - features: word frequencies, metadata 
    - exclude rare words in data set (used by less than 3 users )
- gramatical structure with NLP
- model instances in terms of authors instead of documents 


- Baseline Classifier: Naive Bayes Model ()

"""


"Build Baseline Location Classifier\nApproaches:\n- instance representation in form of 'bag' of words \n    - features: word frequencies, metadata \n    - exclude rare words in data set (used by less than 3 users )\n- gramatical structure with NLP\n- model instances in terms of authors instead of documents \n\n\n- Baseline Classifier: Naive Bayes Model ()\n\n"

In [89]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from os import path, getcwd
from collections import defaultdict
import preprocessor as p
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import re, string
from validators import url
from info_gain.info_gain import info_gain_ratio
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from nltk.stem import SnowballStemmer
# nltk.download('english')
# nltk.download('punkt')

In [28]:
fdir = path.join(getcwd(), "2019S1-proj2-datah")

train_data = "train-raw.tsv"
test_data = "test-raw.tsv"
dev_data = "dev-raw.tsv"

train_fpath = path.join(fdir, train_data)

train = pd.read_csv(train_fpath, sep="\t", index_col="Instance_ID", encoding="utf_8")
train

,Location,Text
Instance_ID,,
1,Melbourne,"\ud83c\udf17 @ Melbourne, Victoria, Australia ..."
2,Melbourne,"@theage Of course it costs more, minimum stand..."
3,Brisbane,Hope people make just as much noise as they di...
4,Perth,Pouring the perfect Prosecco \ud83e\udd42\ud83...
5,Perth,"$LNY losing traction at 0.014, see this retrac..."
6,Melbourne,\u0e44\u0e21\u0e48\u0e44\u0e2b\u0e27\u0e41\u0e...
7,Melbourne,@ashleighjayy_ Me @ this bitch https://t.co/8J...
8,Melbourne,@AnaOLFan I \u2764\ufe0f you - I could never b...
9,Sydney,@ihatejoelkim Welcome to Australia! Hoping you...


In [29]:
inputs = ['Text']
output = 'Location'
x = train[inputs]
y = train[output]

In [104]:
class GeoTagger:
    _FEATURE_SELECTION = ["info_gain_ratio", "word_locality_heuristic", 'chi_squared', 'mutial_info']
    _VOTING_STRATEGY = ["simple_voting", "bagging", "stacking", "random_forest", "boosting"]
    _CLASSIFIERS = ["MultinomialNB", "SVM", "SemiSupervised", "KNN"]
                          
    def __init__(self, classifier_set=["MultinomialNB"], voting_strategy="simple_voting", feature_selection="IGR", seed="12345"):
        self.exclude = set()
        self.classifier_set = classifier_set # MultinomialNB, SVM
        self.stopwords = set(stopwords.words('english'))
        self.stemmer = SnowballStemmer('english')
        np.random.seed(seed)
        
    def train(self, x, y):
        self.classes = y.unique()
        x = self.preprocess(x, train=True)
        return x
    
    def test(self, x, y):
#         self.preprocess(x, y)
        pass
    
    def predict(self, x):
#         self.preprocess(x, y)
        pass
    
    def evaluate(self, x, y):
#         y_pred = self.predict(y)
#         cross validation
        pass
    
    def preprocess(self, x, train=False):
        """
         - Filter rare words (urls, typos rare names, punctuation symbols)
         - calculate word frequencies 
         - metadata
        """
#         if train:
#             feature_weight(x)
#             feature_selection(x)
#       self.class_feature_sets = {label: defaultdict() for label in self.classes}
        x = self.filter(x)
        return x
    
    def filter(self, x):
        """
        https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf
        """
        x.loc[x.index.values, 'Text'] = x['Text'].apply(self.filter_tweet)
        return x        
        
    def feature_selection(self, x):
        """
        (1) Information Gain Ratio (IGR) - across all states S, is 
            defined as the ratio between its information gain value IG, 
            which measures the decrease in class entropy H that w brings,
            and its intrinsic entropy IV, which measures the entropy of 
            the presence versus the absence of that word
            
        (2) Word Locality Heuristic (WLH) - promotes words primarily 
            associated with one location. measure the probability of 
            a word occurring in a state, divided by its probability to 
            appear in any state. Then, for a given word w, we define the 
            WLH as the maximum such probability across all the states S
        """
        if self.feature_selection == "IGR":
#             return information_gain_ratio(x)
             return
        elif self.feature_selection == "WLH":
            return word_locality_weight(x)
#         elif seld
        else:
            print("Invalid Feature Selection method: {}. Choose one of \
            ({})".format(self.feature_selection, ", ".join(GeoTagger._FEATURE_SELECTION)))
        
    def word_locality_weight(self, x):
        """
        calculate frequencies of data 
        Measure frequency and divide by sum of freqencies of all words
        """
    
    def information_gain_ratio(self, x):
#         return info_gain_ratio
        pass
        
    def filter_tweet(self, tweet):
        filtered = [self.filter_word(w) for w in tweet.split() if w not in self.stopwords]
        return ' '.join(filtered)
        
    def filter_word(self, word):
        word = word.lower()
        # extract keywords from hashtag 
        if self._is_hyperlink(word):
            print(word)
            return ''
        elif self._is_hashtag(word):
            word = self._process_hashtag(word)
        # potentially cross-reference individuals mentioned? or discard
        elif self._is_mention(word):
            word = self._process_mention(word)
        # remove ascii characters 
        else:
            word = self._ascii_to_unicode(word)
            word = self._word_stem(word)
            # .decode("unicode_escape").encode('utf-8')
            word = re.sub(r'[^\w\s]','', word)
#         print(word)
        return word
                
    def _is_hashtag(self, word):
        if len(word) == 0:
            return False
        return word[0] == "#"
    
    def _is_mention(self, word):
        if len(word) == 0:
            return False
        return word[0] == "@"
    
    def _is_hyperlink(self, word):
        return url(word)
    
    def _process_hashtag(self, word):
        return word[1:]
    
    def _process_mention(self, word):
        return word[1:]
    
    def _ascii_to_unicode(self, word):
        for uescape in re.findall(r'(\\u[0-9a-z]{4})', word):
            try:
#                 print(uescape.encode('utf-8').decode('unicode-escape'), type(uescape.encode('utf-8').decode('unicode-escape')))
#                 word = re.sub(uescape, uescape.encode('utf-8'), word)
#                 print(word)
#                 print(uescape, type(uescape))
                word = re.sub(uescape, ' ', word)  
            except UnicodeDecodeError:
                print("Failed to decode: {}".format(uescape))
        return word
    
    def _word_stem(self, word):
        return self.stemmer.stem(word)
    

In [105]:
gt = GeoTagger()
x = gt.train(x, y)
print(x['Text'])

Instance_ID
1                    ud83cudf17 melbourn victoria australia
2         theag cour cost minimum standard must anyth 50...
3         hope peopl make much noi tpj shot definit dog ...
4         pour perfect prosecco ud83eudd42ud83cudf7 ud83...
5         lni lose traction 0014 see retrac back 0011 00...
6         u0e44u0e21u0e48u0e44u0e2bu0e27u0e41u0e25u0e49u...
7                                       ashleighjayy_ bitch
8                 anaolfan u2764ufe0f could never offend xx
9         ihatejoelkim welcom australia hope wonder time...
10        mea evolv confer markbouri session learnt wiza...
11                                 first tast winter autumn
12        u0643u064au0641 u064au0634u0648u0641 u0627u063...
13        selfiesunday lost tabbi amp whitecaramel cat m...
14                          artsdesir give credit look incr
15        pringster78 chadderbox2018 kind opposit love u...
16                     gramercypark itu2019 great isnu2019t
17                          

/Users/ninaaverill/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
